In [1]:
import gc #For manage garbage 

In [2]:
import sys


In [3]:
import torch
import datasets
import numpy as np
import pandas as pd
import re
import string


from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import precision_score, recall_score, f1_score



import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import time
import math
import nltk
import random
import os

from typing import List, Tuple

from torch.nn.utils import clip_grad_norm_
from transformers import AdamW
import inspect




import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, MBartConfig
from nltk.translate import bleu_score
from datasets import load_dataset
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from transformers import default_data_collator
#from rouge_score import rouge_scorer
#from nltk.translate.meteor_score import meteor_score
nltk.download('wordnet')
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device ", device)


# torch.cuda.set_per_process_memory_fraction(0.8) 
# %env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# from torch.amp import GradScaler, autocast
 #import gc # Add_to_decrease_memory_for_Collab
# %env CUDA_LAUNCH_BLOCKING=1


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
device  cuda


In [4]:
import transformers
print(torch.__version__)
print(transformers.__version__)

2.4.1+cu121
4.44.2


In [5]:
def print_gpu_memory_usage(step):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024**3)
        reserved = torch.cuda.memory_reserved() / (1024**3)
        free = torch.cuda.get_device_properties(0).total_memory / (1024**3) - reserved
        print(f"{step}: Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB, Free: {free:.2f} GB")
    else:
        print(f"{step}: CUDA not available")


def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

print_gpu_memory_usage("Before loading data")


#dataset_dir = '/kaggle/input/shifts-dataset/'
dataset_dir = '/kaggle/input/shifts-eval/eval-data/'


en_file = os.path.join(dataset_dir, 'reddit_eval.en')
ru_file = os.path.join(dataset_dir, 'reddit_eval.ru')
meta_file = os.path.join(dataset_dir, 'reddit_eval_meta.tsv')



en_lines = read_file(en_file)
ru_lines = read_file(ru_file)


meta_df = pd.read_csv(meta_file, delim_whitespace=True)


assert len(en_lines) == len(ru_lines) == len(meta_df), "Количество строк в файлах не совпадает"


data = []
meta_df.columns = [col.lower() for col in meta_df.columns]

print("Названия столбцов:", meta_df.columns)
print("Первые строки DataFrame:")
print(meta_df.head())

print(f"Количество строк в en_file: {len(en_lines)}")
print(f"Количество строк в ru_file: {len(ru_lines)}")
print(f"Количество строк в meta_file: {len(meta_df)}")


for en, ru in zip(en_lines, ru_lines):
    data.append({
        'en': en.strip(),
        'ru': ru.strip()
    })


df = pd.DataFrame(data)


for i in range(0, 10):
    if i < len(df):
        print(f"\nRow {i+1}:")
        print(f"English: {df.iloc[i]['en']}")
        print(f"Russian: {df.iloc[i]['ru']}")


#print(df.head()) 
#print(df.info()) 

# print(df['length_diff'].describe())
# print(df['is_exact_match'].value_counts())

# df = pd.DataFrame(pairs, columns=['en', 'ru']) # Normal dataset


#check_dataset(data, num_samples=10)  

print_gpu_memory_usage("After loading data")


# en_lines = [
#     "Please, go to the nearest store.",
#     "Good morning, how are you doing?",
#     "This is an example of a longer sentence for testing purposes."
# ]

dataset = datasets.Dataset.from_pandas(df)


# train_size = int(len(dataset) * 0.8)
# eval_size = len(dataset) - train_size

# dataset_train = dataset.select(range(train_size))
# dataset_eval = dataset.select(range(train_size, len(dataset))

print_gpu_memory_usage("After loading data")

dataset = datasets.Dataset.from_pandas(df)

dataset_train = dataset.select(range(0, 60))
dataset_eval = dataset.select(range(0, 60))

print(f"Train dataset size: {len(dataset_train)}")
print(f"Eval dataset size: {len(dataset_eval)}")

Before loading data: Allocated: 0.00 GB, Reserved: 0.00 GB, Free: 15.89 GB
Названия столбцов: Index(['punct', 'missp', 'cap', 'flu', 'slang', 'emoji', 'tags'], dtype='object')
Первые строки DataFrame:
   punct  missp  cap  flu  slang  emoji  tags
0      0      1    0    0      1      0   0.0
1      0      0    0    0      0      0   0.0
2      1      0    0    0      1      0   0.0
3      0      1    0    0      0      0   0.0
4      1      0    0    0      0      0   0.0
Количество строк в en_file: 3065
Количество строк в ru_file: 3065
Количество строк в meta_file: 3065

Row 1:
English: I’ll prolly be picking up the i Sport!
Russian: Наверное, я возьму iSport!

Row 2:
English: My mind just melted after seeing this, I really want to be able to look like you someday.
Russian: Я просто растаял, увидев это, мне бы реально хотелось когда-нибудь выглядеть так, как ты.

Row 3:
English: Such a cute clitty, and pussy you have
Russian: У тебя такие милые клиторок и киска.

Row 4:
English: Spide

In [6]:
tokenizer = MBart50Tokenizer.from_pretrained('facebook/mbart-large-50', src_lang="en_XX", tgt_lang="ru_RU", return_tensors="pt")

def collate_fn(batch):
    return default_data_collator(batch)

def preprocess_function(examples):
    inputs = examples['en']
    targets = examples['ru']
    model_inputs = tokenizer(inputs, text_target=targets,
                                padding=True, truncation=True)
    return model_inputs
    
model_inputs_train = dataset_train.map(preprocess_function,
                                            batched=True)
model_inputs_test = dataset_eval.map(preprocess_function,
                                            batched=True)

train_loader = DataLoader(dataset=model_inputs_train,
                            collate_fn=collate_fn, batch_size=10, pin_memory=True, drop_last=True)
validation_loader = DataLoader(dataset=model_inputs_test,
                                collate_fn=collate_fn, batch_size=10, pin_memory=True, drop_last=True)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [7]:

def extract_attention_uncertainty(text, mbart_model, tokenizer, device, batch_size=8):

    uncertainty_scores = []

    for i in range(0, len(text), batch_size):
        batch = text[i:i + batch_size]
        batch_inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()} 

        # print_gpu_memory_usage(f"Before forward pass for batch {i // batch_size + 1}")
        with torch.no_grad():
            outputs = mbart_model(**batch_inputs, output_attentions=True)
            # print_gpu_memory_usage(f"After forward pass for batch {i // batch_size + 1}")

        if not outputs.encoder_attentions:
            raise ValueError("Модель не вернула attention веса")

        attentions = outputs.encoder_attentions
        attentions = [attn for attn in attentions if attn is not None] 

        if not attentions:
            continue

        avg_attention = torch.mean(torch.stack(attentions), dim=(0, 1))  # (batch, seq_len, seq_len)

        attention_entropy = -torch.sum(avg_attention * torch.log(torch.clamp(avg_attention, min=1e-6)), dim=-1)  # (batch, seq_len)

        batch_uncertainty_scores = attention_entropy.mean(dim=-1).cpu().numpy()
        uncertainty_scores.extend(batch_uncertainty_scores)

        del batch_inputs, outputs, attentions, avg_attention, attention_entropy
        torch.cuda.empty_cache()

    # print_gpu_memory_usage("After processing all batches")
    return uncertainty_scores


# ==========================================
# 3.
# ==========================================

class QEDataset(Dataset):
    def __init__(self, en_texts, ru_texts, uncertainty_scores, tokenizer, mbart_model, device):

        self.en_texts = en_texts
        self.ru_texts = ru_texts
        self.uncertainty_scores = uncertainty_scores
        self.tokenizer = tokenizer
        self.mbart_model = mbart_model
        self.device = device

    def __len__(self):
        return len(self.en_texts)

    def __getitem__(self, idx):
        en_text = self.en_texts[idx]
        ru_text = self.ru_texts[idx]
        uncertainty = self.uncertainty_scores[idx]

        # Convert uncertainty to tensor if it's a float or numpy float
        if isinstance(uncertainty, (float, np.float32, np.float64)):
            uncertainty = torch.tensor(uncertainty, dtype=torch.float32)
        elif isinstance(uncertainty, torch.Tensor):
            uncertainty = uncertainty.float()
        else:
            raise TypeError("Uncertainty score must be a float or a torch.Tensor.")

        return en_text, ru_text, uncertainty

    def collate_fn(self, batch):
        en_texts = [item[0] for item in batch]
        ru_texts = [item[1] for item in batch]
        uncertainties = torch.stack([item[2] for item in batch])

        return en_texts, ru_texts, uncertainties



class QENetWithUncertainty(nn.Module):
    def __init__(self, hidden_dim=1024, uncertainty_dim=5, output_dim=1):

        super(QENetWithUncertainty, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim + uncertainty_dim, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, features, uncertainties):
  
        x = torch.cat([features, uncertainties], dim=1)
        return self.fc(x)


# ==========================================
# 5.  
# ==========================================
def extract_mbart_features(texts, mbart_model, tokenizer, device, features_file="mbart_features.pt"):

    if os.path.exists(features_file):
        saved_features = torch.load(features_file, map_location=device)
        if saved_features.size(0) == len(texts):
            return saved_features

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = mbart_model(**inputs, output_hidden_states=True, return_dict=True)
        last_hidden = outputs.encoder_last_hidden_state  # (batch_size, seq_len, hidden_dim)

    features = last_hidden.mean(dim=1)  # (batch_size, hidden_dim)
    
    # Save
    torch.save(features, features_file)
    # print(f"Features saved to {features_file}.")

    return features


def entropy_from_logits(logits):

    probs = torch.softmax(logits, dim=-1) 
    entropy = -(probs * torch.log(probs.clamp(min=1e-10))).sum(dim=-1).mean(dim=1)
    return entropy


# def refine_translation_with_qe(
#     text,
#     mbart_model,
#     qe_model,
#     tokenizer,
#     device,
#     mbart_uncertainty_weight=1.0,
#     qe_uncertainty_weight=1.0,
#     desired_mbart_uncertainty=0.2,
#     desired_qe_uncertainty=0.2,
#     fine_tune_steps=10,
#     learning_rate=1e-5,
#     threshold=0.7
# ):
#     
#     
#     

#    
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

#    
#     with torch.no_grad():
#         gen_ids = mbart_model.generate(
#             **inputs,
#             max_length=200,
#             num_beams=5,
#             do_sample=False,
#             forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
#             repetition_penalty=1.1
#         )
#     translation = tokenizer.decode(gen_ids[0], skip_special_tokens=True)

#     
#     mbart_model.eval()
#     with torch.no_grad():
#         outputs = mbart_model(
#             **inputs,
#             output_attentions=True,
#             output_hidden_states=True,
#             return_dict=True
#         )

#    
#     # last_hidden = outputs.encoder_last_hidden_state  # shape: [1, seq_len, hidden_dim]
#     # features = last_hidden.mean(dim=1)  # shape: [1, hidden_dim]

#     features = extract_mbart_features(text, mbart_model, tokenizer, device)

#     
#     attentions = outputs.encoder_attentions  # список тензоров: [num_layers] каждый shape: [1, num_heads, seq_len, seq_len]
#     avg_attention = torch.mean(torch.stack(attentions), dim=(0, 1))  # shape: [1, seq_len, seq_len]
#     # attn_entropy = -torch.sum(avg_attention * torch.log(avg_attention + 1e-9), dim=-1).mean()

#     attn_entropy = extract_attention_uncertainty([text], mbart_model, tokenizer, device, batch_size=8)

#     print()
#     print()
#     print("attn_entropy ", attn_entropy) #mc_dropout_val  tensor([0.7919], device='cuda:0')

#     print()
#     print()

#     
#     attn_variance = avg_attention.var(dim=-1)  # shape: [1, seq_len]
#     attn_variance_mean = attn_variance.mean()  # скаляр
#     attn_variance_first = attn_variance[0].mean().item() 

#    
#    
#     
#     logits = outputs.logits if hasattr(outputs, "logits") else None
#     if logits is not None:
#         # energy_val = energy_score_compute(logits).mean().item()
#         entropy_val = entropy_from_logits(logits).mean().item()
#     else:
#         # energy_val = 0.0
#         entropy_val = 0.0

#     
#     print()
#     print(f"entropy_val: {entropy_from_logits(logits).mean().item()}")
#     print()
#     print()
    
#     
#     
    
#     (translated_ids, scores_dropout,
#      entropy_scores_demo, attention_variances_demo,
#      hidden_variances_demo) = mc_dropout_inference(model, inputs, n=10, batch_size=2, device=device)


#     mc_dropout_val = scores_dropout
#     print("mc_dropout_val ", mc_dropout_val) #mc_dropout_val  tensor([0.7919], device='cuda:0')
#     # print()
#     # print()

    
#     
#     uncertainty_values = [
#         mc_dropout_val,
#         # energy_val,
#         float(entropy_val),
#         float(attn_variance_mean),
#         float(attn_variance_first)
#     ]
#     uncertainty_tensor = torch.tensor([uncertainty_values], dtype=torch.float32).to(device)

#    
#     with torch.no_grad():
#         qe_uncertainty = qe_model(features, uncertainty_tensor)

#     print("qe_uncertainty ", qe_uncertainty) #qe_uncertainty  tensor([[1.8887]], device='cuda:0')
#     print()
#     print()
#     
#     combined_uncertainty = 0.7 * float(attn_entropy[0]) + 0.3 * qe_uncertainty.item()

#     # print(f"Text: {text}")
#     # print(f"Translation before fine-tuning: {translation}")
#     print(f"MBart attention-entropy: {float(attn_entropy[0]):.4f}, QE uncertainty: {qe_uncertainty.item():.4f}")
#     print(f"Combined uncertainty: {combined_uncertainty:.4f}")

#     
#     if combined_uncertainty > threshold:
#         print("High uncertainty detected. Initiating fine-tuning.")
#         
#         mbart_model.train()
    
#         
#         for p in mbart_model.parameters():
#             p.requires_grad = False
    
#         
#         for layer in mbart_model.model.encoder.layers[-3:]:
#             for p in layer.parameters():
#                 p.requires_grad = True
    
#         
#         for layer in mbart_model.model.decoder.layers:
#             for p in layer.encoder_attn.parameters():
#                 p.requires_grad = True
#             for p in layer.encoder_attn_layer_norm.parameters():
#                 p.requires_grad = True

#         
#         qe_model.eval()
#         for p in qe_model.parameters():
#             p.requires_grad = False

#      
#         # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, mbart_model.parameters()), lr=learning_rate)
#         trainable_params = [p for p in mbart_model.parameters() if p.requires_grad]
#         optimizer = torch.optim.Adam(trainable_params, lr=learning_rate)

        
#         criterion = torch.nn.MSELoss()

#         for step in range(fine_tune_steps):
#             optimizer.zero_grad(set_to_none=True)
        
#             out = mbart_model(
#                 **inputs,
#                 output_attentions=True,
#                 output_hidden_states=True,
#                 return_dict=True
#             )
        
#            
#             attn_entropy_ft = (
#                 -torch.stack(out.encoder_attentions).mean(dim=(0, 1))
#                  .mul(torch.log(torch.stack(out.encoder_attentions).mean(dim=(0, 1)) + 1e-9))
#                  .sum(dim=-1).mean()
#             )
        
#            
#             last_h = out.encoder_last_hidden_state.mean(dim=1)
        
#             uncertain_tensor = torch.tensor(
#                 [
#                     [
#                         attn_entropy_ft.detach(),     # tensor, градиент не нужен
#                         entropy_val,                  # фиксированное
#                         attn_variance_mean,           # фиксированное
#                         attn_variance_first           # фиксированное
#                     ]
#                 ],
#                 dtype=torch.float32,
#                 device=device
#             )
        
#             pred_qe_unc = qe_model(last_h, uncertain_tensor)
        
#             loss_mbart = criterion(
#                 attn_entropy_ft.unsqueeze(0),
#                 torch.as_tensor([desired_mbart_uncertainty], device=device)
#             )
#             loss_qe = criterion(
#                 pred_qe_unc,
#                 torch.as_tensor([[desired_qe_uncertainty]], device=device)
#             )
        
#             loss = (mbart_uncertainty_weight * loss_mbart +
#                     qe_uncertainty_weight * loss_qe)
        
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)
#             optimizer.step()
        
#             print(f'  Fine-tune step {step+1}/{fine_tune_steps}, Loss = {loss.item():.4f}')


#         mbart_model.eval()

#        
#         with torch.no_grad():
#             # refined_ids = model.generate(
#             #         **inputs,
#             #         max_length=200,
#             #         num_beams=5,
#             #         do_sample=False,
#             #         forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
#             #         repetition_penalty=1.1
#             #     )


#             # refined_ids = mbart_model.generate(
#             #     **inputs,
#             #     max_length=200,
#             #     num_beams=5,         # ← тот же search, что и в baseline
#             #     do_sample=False,
#             #     forced_bos_token_id=tokenizer.lang_code_to_id['ru_RU'],
#             #     repetition_penalty=1.1
#             # )
                
#             refined_ids = mbart_model.generate(
#                 **inputs,
#                 max_length=200,
#                 num_beams=4,
#                 do_sample=True,  # <== ключевой флаг
#                 top_k=50,
#                 top_p=0.9,
#                 temperature=0.8,
#                 num_return_sequences=1,
#                 forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
#                 repetition_penalty=1.1
#             )
#         translation = tokenizer.decode(refined_ids[0], skip_special_tokens=True)
#         #print(f"Refined translation after fine-tuning: {translation}")
#     else:
#         print("Uncertainty is within acceptable limits. No fine-tuning performed.")

#     return translation



RISK_BUFFER: list[tuple[str, str]] = []     # (src, tgt)

def refine_translation_with_qe(
        src_text: str,
        tgt_text: str,
        mbart_model,
        qe_model,
        tokenizer,
        device,
        *,
        threshold: float = 0.9,
        buffer_size: int = 64,
        fine_tune_epochs: int = 2,
        batch_size: int = 8,
        learning_rate: float = 1e-5,
        label_smoothing: float = 0.1,
        verbose: bool = True
) -> str:
   
    decode_kwargs = dict(
        max_length=200, num_beams=4, do_sample=True,
        top_k=50, top_p=0.9, temperature=0.8,
        num_return_sequences=1,
        forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
        repetition_penalty=1.1
    )

    
    mbart_model.eval()
    inputs = tokenizer(src_text, return_tensors="pt").to(device)
    with torch.no_grad():
        base_ids = mbart_model.generate(**inputs, **decode_kwargs)
    base_translation = tokenizer.decode(base_ids[0], skip_special_tokens=True)

    
    attn_entropy = extract_attention_uncertainty([src_text], mbart_model, tokenizer, device)[0]
    features = extract_mbart_features(src_text, mbart_model, tokenizer, device)
    uncertainty_tensor = torch.tensor([[0.,0.,0.,0.]], device=device)  # если другие фичи не нужны
    with torch.no_grad():
        qe_uncert = qe_model(features, uncertainty_tensor).item()
    risk = 0.7 * attn_entropy.item() + 0.3 * qe_uncert
    needs_ft = risk > threshold

    
    with torch.no_grad():
        temp_ids = mbart_model.generate(**inputs, **decode_kwargs)
    temp_refined = tokenizer.decode(temp_ids[0], skip_special_tokens=True)

    
    if verbose:
        print("=" * 80)
        print(f"SRC       : {src_text}")
        print(f"TGT       : {tgt_text}")
        print(f"BASE_MBART: {base_translation}")
        print(f"attention_entropy = {attn_entropy.item():.4f} | "
              f"qe_uncert = {qe_uncert:.4f} | "
              f"combined_risk = {risk:.4f}  -->  "
              f"{'ADD_TO_BUFFER' if needs_ft else 'OK'}")

        
        ref_tok  = tokenizer.tokenize(tgt_text)
        base_tok = tokenizer.tokenize(base_translation)
        refn_tok = tokenizer.tokenize(temp_refined)

       
        def safe_sentence_bleu(ref_tok, hyp_tok):
            if not ref_tok or not hyp_tok:
                return 0.0
            smoothing = SmoothingFunction().method4
            try:
                return sentence_bleu([ref_tok], hyp_tok, smoothing_function=smoothing)
            except ZeroDivisionError:
                return 0.0

        bleu_base = safe_sentence_bleu(ref_tok, base_tok)
        bleu_ref  = safe_sentence_bleu(ref_tok, refn_tok)
        print(f"BLEU       | base: {bleu_base:.3f}  refined: {bleu_ref:.3f}")

        
        def safe_prf(true_tok, pred_tok):
            if not true_tok or not pred_tok:
                return 0.0, 0.0, 0.0
            
            L = min(len(true_tok), len(pred_tok))
            t, p = true_tok[:L], pred_tok[:L]
            p_score = precision_score(t, p, average='micro', zero_division=0)
            r_score = recall_score(   t, p, average='micro', zero_division=0)
            f1_s    = f1_score(      t, p, average='micro', zero_division=0)
            return p_score, r_score, f1_s

        prec_b, rec_b, f1_b = safe_prf(ref_tok, base_tok)
        prec_r, rec_r, f1_r = safe_prf(ref_tok, refn_tok)
        print(f"Precision    | base: {prec_b:.3f}  refined: {prec_r:.3f}")
        print(f"Recall    | base: {rec_b:.3f}  refined: {rec_r:.3f}")
        print(f"F1 Score   | base: {f1_b:.3f}  refined: {f1_r:.3f}")

    
    if needs_ft:
        RISK_BUFFER.append((src_text, tgt_text))
    if len(RISK_BUFFER) >= buffer_size:
        if verbose:
            print(f"\n▶ fine-tuning on {len(RISK_BUFFER)} risky pairs …")
        _run_batch_finetune(
            mbart_model, tokenizer, device, RISK_BUFFER,
            epochs=fine_tune_epochs, bs=batch_size,
            lr=learning_rate, lbl_smth=label_smoothing, verbose=verbose
        )
        RISK_BUFFER.clear()

    
    with torch.no_grad():
        final_ids = mbart_model.generate(**inputs, **decode_kwargs)
    refined_translation = tokenizer.decode(final_ids[0], skip_special_tokens=True)

    if verbose:
        delta = "(unchanged)" if refined_translation == base_translation else "⇒ UPDATED"
        print(f"REFINED_MBART: {refined_translation}  {delta}\n")

    return refined_translation



def _run_batch_finetune(model, tokenizer, device, pairs,
                        *, epochs, bs, lr, lbl_smth, verbose):
    
    for p in model.parameters(): p.requires_grad = False
    for l in model.model.encoder.layers[-3:]:
        for p in l.parameters(): p.requires_grad = True
    for p in model.model.decoder.parameters(): p.requires_grad = True

    optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    ce_loss_fn = torch.nn.CrossEntropyLoss(
        ignore_index=tokenizer.pad_token_id,
        label_smoothing=lbl_smth
    )

    model.train()
    for ep in range(epochs):
        random.shuffle(pairs)
        for i in range(0, len(pairs), bs):
            src_batch, tgt_batch = zip(*pairs[i:i+bs])
            enc = tokenizer(list(src_batch), return_tensors="pt",
                            padding=True, truncation=True).to(device)
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(list(tgt_batch), return_tensors="pt",
                                   padding=True, truncation=True).input_ids.to(device)
            enc["labels"] = labels

            optim.zero_grad()
            out   = model(**enc)
            loss  = out.loss
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optim.step()
        if verbose:
            print(f"  epoch {ep+1}/{epochs}  loss={loss.item():.4f}")
    model.eval()






def hidden_state_variance(enc_outputs):

    last_hidden = enc_outputs.encoder_last_hidden_state 
    variance = last_hidden.var(dim=1)  
    return variance.mean(dim=-1) 

# def energy_score_compute(logits):

#     return -torch.logsumexp(logits, dim=-1).mean(dim=1) 


def compute_attention_variance(attention_weights):

    variance = attention_weights.var(dim=-1) 
    return variance.mean(dim=(1, 2))


def mahalanobis_distance(x, mean, cov):

    if x.dim() != 2 or mean.dim() != 1 or cov.dim() != 2:
        raise ValueError("Неверные размерности входных данных.")
    if x.size(1) != mean.size(0) or cov.size(0) != cov.size(1) or cov.size(0) != mean.size(0):
        raise ValueError("Несоответствие размерностей между x, mean и cov.")

    if torch.isnan(cov).any() or torch.isinf(cov).any():
        raise ValueError("Ковариационная матрица содержит NaN или бесконечные значения.")

    diff = x - mean 


    epsilon = 1e-5 
    cov = cov + epsilon * torch.eye(cov.size(0), device=cov.device)

    inv_cov = pinv_qr(cov)

    left_term = torch.matmul(diff, inv_cov) 
    mahalanobis_sq = torch.sum(left_term * diff, dim=1) 
    distances = torch.sqrt(mahalanobis_sq) 

    return distances



def mutual_information(logits_list):

    if not logits_list:
        raise ValueError("logits_list is empty.")


    first_shape = logits_list[0].shape
    if not all(logits.shape == first_shape for logits in logits_list):
        raise ValueError("All tensors in logits_list must have the same shape.")


    probs = torch.stack([torch.softmax(logits.float(), dim=-1) for logits in logits_list])


    mean_probs = probs.mean(dim=0) 


    epsilon = 1e-6 
    entropy_mean = -(mean_probs * torch.log(mean_probs + epsilon)).sum(dim=-1) 


    mean_entropy = -(probs * torch.log(probs + epsilon)).sum(dim=-1).mean(dim=0) 


    mutual_info = entropy_mean - mean_entropy 


    return mutual_info.mean(dim=-1)  




def cosine_similarity_score(h, h_train):
    cos_sim = torch.nn.functional.cosine_similarity(h, h_train)
    return 1 - cos_sim.mean()




def pad_sequences(sequences, pad_token_id=1):

    max_length = max(seq.size(-1) for seq in sequences)

    padded_seqs = []
    for seq in sequences:
        if seq.dim() == 1:
            seq = seq.unsqueeze(0)

        seq_length = seq.size(1)
        if seq_length < max_length:
            padding = torch.full((seq.size(0), max_length - seq_length), pad_token_id,
                                 dtype=seq.dtype, device=seq.device)
            padded_seq = torch.cat([seq, padding], dim=1)
        else:
            padded_seq = seq[:, :max_length]

        padded_seqs.append(padded_seq)

    return torch.cat(padded_seqs, dim=0)
    
def enable_dropout(model):
    for module in model.modules():
        if isinstance(module, torch.nn.Dropout):
            module.train()

def mc_dropout_inference(model, inputs, n=10, batch_size=2, device="cuda"):

    model.eval()
    enable_dropout(model) 

    num_samples = inputs["input_ids"].size(0)
    predictions, entropy_scores = [], []
    attention_variances, hidden_variances = [], []
    uncertainty_scores = []

    # print(f"Общее количество предложений: {num_samples}")

    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        batch_inputs = {k: v[start:end].to(device) for k, v in inputs.items()}
    
        batch_predictions = []
        batch_entropy_scores = []
        batch_attention_variances = []
        batch_hidden_variances = []
    
        for _ in range(n):
            with torch.no_grad():
                # Генерация перевода
                gen_ids = model.generate(
                    **batch_inputs,
                    max_length=200,
                    num_beams=5,
                    do_sample=False,
                    forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
                    repetition_penalty=1.1
                )
    
                batch_predictions.append(gen_ids.cpu())
    
                # Прямой проход модели
                outputs = model(**batch_inputs, return_dict=True, output_hidden_states=True, output_attentions=True)
                logits1 = outputs.logits
    
                # # Энергетический скор
                # if callable(energy_score_compute):
                #     temp = energy_score_compute(logits1).cpu()
                #     batch_energy_scores.append(temp)
                # else:
                #     print(f"[Warning] energy_score_compute is not callable. Type: {type(energy_score_compute)}")
    
                # Энтропия из логитов
                batch_entropy_scores.append(entropy_from_logits(logits1).cpu())
    
                # Дисперсия скрытых состояний
                batch_hidden_variances.append(hidden_state_variance(outputs).cpu())
    
                # Дисперсия внимания
                if outputs.encoder_attentions:
                    attention_weights = torch.stack(outputs.encoder_attentions, dim=0).mean(dim=0)
                    
                    if callable(compute_attention_variance):
                        batch_attention_variances.append(compute_attention_variance(attention_weights).cpu())
                    else:
                        print(f"[Warning] compute_attention_variance is not callable. Type: {type(compute_attention_variance)}")
                else:
                    batch_attention_variances.append(torch.zeros(logits1.shape[0], device=device))
    
                # Очистка памяти
                del gen_ids, outputs, logits1
                torch.cuda.empty_cache()
                gc.collect()

        batch_predictions = pad_sequences(batch_predictions, pad_token_id=1)

        # batch_energy_scores = torch.stack(batch_energy_scores).mean(dim=0)
        batch_entropy_scores = torch.stack(batch_entropy_scores).mean(dim=0)
        batch_hidden_variances = torch.stack(batch_hidden_variances).mean(dim=0)
        batch_attention_variances = torch.stack(batch_attention_variances).mean(dim=0)

        batch_uncertainty_scores = (batch_entropy_scores + batch_hidden_variances) / 2
        uncertainty_scores.append(batch_uncertainty_scores)

        predictions.append(batch_predictions.mode(dim=0)[0])
        # energy_scores.append(batch_energy_scores)
        entropy_scores.append(batch_entropy_scores)
        attention_variances.append(batch_attention_variances)
        hidden_variances.append(batch_hidden_variances)

        print(f"Batch {start // batch_size + 1}: обработано {batch_uncertainty_scores.shape[0]} предложений")

    model.train()

    predictions = pad_sequences(predictions, pad_token_id=1)
    # energy_scores = torch.cat(energy_scores, dim=0)
    entropy_scores = torch.cat(entropy_scores, dim=0)
    attention_variances = torch.cat(attention_variances, dim=0)
    hidden_variances = torch.cat(hidden_variances, dim=0)
    uncertainty_scores = torch.cat(uncertainty_scores, dim=0)

    return (
        predictions.to(device),
        uncertainty_scores.to(device),
        # energy_scores.to(device),
        entropy_scores.to(device),
        attention_variances.to(device),
        hidden_variances.to(device)
    )

def pinv_qr(A):
    Q, R = torch.qr(A)
    return torch.mm(R.pinverse(), Q.t())






def create_model_with_dropout(dropout_rate):
     config = MBartConfig.from_pretrained('facebook/mbart-large-50-many-to-many-mmt',
                                            dropout=dropout_rate, attention_dropout=dropout_rate)
     config.output_attentions = True  # Включаем return attentions
     model_dir = "/kaggle/input/mbart-large-50-70000to170000-80000to100000-5epochs/pytorch/default/1/results/to/save/model_dropout_result"
     model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt', config=config)
     # model = MBartForConditionalGeneration.from_pretrained("path/to/save/model_dropout_0.1", config=config)
     # model = MBartForConditionalGeneration.from_pretrained(model_dir, config=config)
     return model

In [8]:


print_gpu_memory_usage("Before loading model")

model = create_model_with_dropout(0.3)
model.to(device)

print_gpu_memory_usage("After loading model")

ru_texts_demo = []

ru_texts_file = "ru_texts_demo.txt"

if os.path.exists(ru_texts_file):
    with open(ru_texts_file, "r", encoding="utf-8") as f:
        ru_texts_demo = [line.strip() for line in f.readlines()]
    if len(ru_texts_demo) != len(en_lines):
        print("File found, but the number of lines does not match. Translations will be regenerated.")
        ru_texts_demo = []
else:
    ru_texts_demo = []

print("Number of lines in en_lines:", len(en_lines))

if not ru_texts_demo:
    for i, txt in enumerate(en_lines[:100]):

        inputs = tokenizer(txt, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}


        with torch.no_grad():
            gen_ids = model.generate(
                **inputs,
                max_length=200,
                num_beams=5,  
                do_sample=False, 
                forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"], 
                repetition_penalty=1.1
            )

        ru_translation = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        ru_texts_demo.append(ru_translation)

        input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)


        if i % 50 == 0:
            print(f"Generated {i} sentences")

        del inputs, gen_ids

    gc.collect()

    with open(ru_texts_file, "w", encoding="utf-8") as f:
        for line in ru_texts_demo:
            f.write(line + "\n")

    #####-----------------------------------------------


if ru_texts_demo:
    with open(ru_texts_file, "w", encoding="utf-8") as f:
        f.write("\n".join(ru_texts_demo))
    print(f"Translations saved to {ru_texts_file}")
else:
    print(f"File {ru_texts_file} found, translations loaded.")


# print_gpu_memory_usage("After processing lines")

indices = random.sample(range(len(ru_texts_demo)), min(5, len(ru_texts_demo)))


for i in indices:
    print(f"Input sentence: {en_lines[i]}")
    print(f"Generated MBart translation: {ru_texts_demo[i]}\n")


scores_demo = extract_attention_uncertainty(en_lines[:100], model, tokenizer, device)


inputs = tokenizer(en_lines[:10], padding=True, truncation=True, return_tensors='pt').to(device)


(translated_ids, scores_dropout,
 entropy_scores_demo, attention_variances_demo,
 hidden_variances_demo) = mc_dropout_inference(model, inputs, n=10, batch_size=2, device=device)


ru_texts_demo_mcdropout = tokenizer.batch_decode(
    [seq for batch in translated_ids.tolist() for seq in batch],  
    skip_special_tokens=True
)


dataset_demo = QEDataset(en_lines[:100], ru_texts_demo, scores_demo, tokenizer, model, device)
dataloader_demo = DataLoader(dataset_demo, batch_size=2, shuffle=True, collate_fn=dataset_demo.collate_fn)


print(f"Size of en_lines[:100]: {len(en_lines[:100])}")
print(f"Size of ru_texts_demo_mcdropout: {len(ru_texts_demo_mcdropout)}")
print(f"Size of scores_dropout: {len(scores_dropout)}")


min_size = min(len(en_lines[:100]), len(ru_texts_demo_mcdropout), len(scores_dropout))

en_lines_trimmed = en_lines[:min_size]
ru_texts_demo_mcdropout_trimmed = ru_texts_demo_mcdropout[:min_size]
scores_dropout_trimmed = scores_dropout[:min_size]


dataset_demo_mcdropout = QEDataset(
    en_lines_trimmed,
    ru_texts_demo_mcdropout_trimmed,
    scores_dropout_trimmed,
    tokenizer,
    model,
    device
)

dataloader_demo_mcdropout = DataLoader(
    dataset_demo_mcdropout,
    batch_size=1,
    shuffle=True,
    collate_fn=dataset_demo_mcdropout.collate_fn
)

print(f"Updated: dataset_demo_mcdropout now contains {len(dataset_demo_mcdropout)} examples.")


Before loading model: Allocated: 0.00 GB, Reserved: 0.00 GB, Free: 15.89 GB


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

After loading model: Allocated: 2.28 GB, Reserved: 2.29 GB, Free: 13.60 GB
Number of lines in en_lines: 3065
Generated 0 sentences
Generated 50 sentences
Translations saved to ru_texts_demo.txt
Input sentence: This project offers a collateralization rate of 105%.

Generated MBart translation: Этот проект предлагает процентную гарантию в размере 105 %.

Input sentence: You will definitely make more characters in the future.

Generated MBart translation: Вы обязательно будете делать больше персонажей в будущем.

Input sentence: Bless Unleashed, Black Desert, Neverwinter, FF14(Playstation), Elder Scrolls Online, Phantasy Star Online 2(My fave), Trove, Skyforge, DC online, Star Trek Online(Traaaash).

Generated MBart translation: Bless Unleashed, Black Desert, Neverwinter, FF14(Playstation), Elder Scrolls Online, Phantasy Star Online 2(My fave), Trove, Skyforge, DC online, Star Trek Online(Traaaash).

Input sentence: ambery hazel

Generated MBart translation: янтарная медь

Input sentence:

In [9]:
# ==========================================
# 7. Обучение QE-модели
# ==========================================
qe_model = QENetWithUncertainty(hidden_dim=1024, uncertainty_dim=4).to(device)
optimizer = optim.Adam(qe_model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

num_epochs = 3

print_gpu_memory_usage("Before_QE")

# Обучение QE
for epoch in range(num_epochs):
    qe_model.train()
    total_loss = 0
    
    for i, (en_batch, ru_batch, uncertainty_batch) in enumerate(dataloader_demo_mcdropout):

        features = extract_mbart_features(en_batch, model, tokenizer, device)

        uncertainty_features = []
        for j in range(len(en_batch)):
            dataset_index = i * dataloader_demo_mcdropout.batch_size + j 


            score_value_mcdropout = scores_dropout[dataset_index].item() if dataset_index < len(scores_dropout) else 0.0
            #energy_score = energy_scores_demo[dataset_index].item() if dataset_index < len(energy_scores_demo) else 0.0
            entropy_score = entropy_scores_demo[dataset_index].item() if dataset_index < len(entropy_scores_demo) else 0.0
            attention_variance_mean = attention_variances_demo[dataset_index].mean().item() if dataset_index < len(attention_variances_demo) else 0.0


            if dataset_index < len(attention_variances_demo):
                attention_variance = attention_variances_demo[dataset_index]

                if attention_variance.dim() == 0:
                    attention_variance_specific = attention_variance.item()
                else:
                    attention_variance_specific = attention_variance[0].item()
            else:
                attention_variance_specific = 0.0


            uncertainty_values = [
                score_value_mcdropout,
                #energy_score,
                entropy_score,
                attention_variance_mean,
                attention_variance_specific
            ]
            uncertainty_features.append(uncertainty_values)


        uncertainty_features_tensor = torch.tensor(uncertainty_features, dtype=torch.float32).to(device)


        pred_uncertainties = qe_model(features, uncertainty_features_tensor)


        pred_uncertainties = pred_uncertainties.squeeze(-1) 
        target = uncertainty_batch.to(device).view(-1)       
        loss = criterion(pred_uncertainties, target)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader_demo_mcdropout)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

print_gpu_memory_usage("After_QE")

Before_QE: Allocated: 2.29 GB, Reserved: 2.31 GB, Free: 13.58 GB
Epoch 1/3, Loss: 1.5553
Epoch 2/3, Loss: 0.4600
Epoch 3/3, Loss: 0.1359
After_QE: Allocated: 2.30 GB, Reserved: 2.39 GB, Free: 13.49 GB


In [10]:

ROOT = "/kaggle/input/shifts-eval/eval-data/"

file_en   = ROOT + "reddit_eval.en"
file_ru   = ROOT + "reddit_eval.ru"
file_meta = ROOT + "reddit_eval_meta.tsv"


with open(file_en, encoding="utf-8") as f:
    en_lines = [l.rstrip("\n") for l in f]
with open(file_ru, encoding="utf-8") as f:
    ru_lines = [l.rstrip("\n") for l in f]

meta_df = pd.read_csv(file_meta, sep="\t")
assert len(en_lines) == len(ru_lines) == len(meta_df)

df1 = pd.DataFrame({
    "en":   en_lines,
    "ru":   ru_lines,
    **{c: meta_df[c] for c in meta_df.columns}
})


SLANG  = r"\b(bruh|lol|omg|idk|smh|nah|irl|wanna|ain't|ya|tho)\b"
EMOJI  = "[\U00010000-\U0010ffff]"
NONENG = f"[^{string.ascii_letters} ']"              

def is_hard(row) -> bool:
    txt   = row["en"]
    words = txt.split()

   
    if row.get("missp", False) or row.get("slang", False) or row.get("emoji", False):
        return True

    if len(words) > 25 or len(words) < 3:
        return True
    if re.search(SLANG, txt.lower()) or re.search(EMOJI, txt):
        return True
    if len(re.findall(NONENG, txt)) / max(len(txt),1) > .15:
        return True
    return False

df1["difficulty"] = df1.apply(is_hard, axis=1).map({True:"hard", False:"easy"})
easy_df, hard_df = [g for _, g in df1.groupby("difficulty")]

print(f"easy = {len(easy_df)}  |  hard = {len(hard_df)}")


easy_100 = easy_df.iloc[:100][["en","ru","difficulty"]]
hard_100 = hard_df.iloc[:100][["en","ru","difficulty"]]

print("Easy samples:")
display(easy_100)

print("Hard samples:")
display(hard_100)


# tokenizer.src_lang = "en_XX"
# FORCED = tokenizer.lang_code_to_id["ru_RU"]

# def batch_translate(model, sentences: List[str], batch_size: int = 16) -> List[str]:
#     
#     results = []
#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i : i + batch_size]
#         enc = tokenizer(batch,
#                         return_tensors="pt",
#                         padding=True,
#                         truncation=True,
#                         max_length=128).to(device)
#         with torch.no_grad():
#             out = model.generate(
#                 **enc,
#                 max_length=200,
#                 num_beams=5,
#                 forced_bos_token_id=FORCED
#             )
#         results.extend(tokenizer.batch_decode(out, skip_special_tokens=True))
#         
#         del enc, out
#         torch.cuda.empty_cache()
#     return results


# def eval_split(split_df):
#     en_texts = split_df["en"].tolist()
#     refs     = split_df["ru"].tolist()

#    
#     with torch.no_grad():
#         base_pred = batch_translate(model, en_texts, batch_size=16)

#     
#     ours_pred = []
#     for i, src in enumerate(en_texts):
#         pred = refine_translation_with_qe(
#             src_text=src,
#             tgt_text=refs[i],
#             mbart_model=model,
#             qe_model=qe_model,
#             tokenizer=tokenizer,
#             device=device
#         )
#         ours_pred.append(pred)
#         torch.cuda.empty_cache()

#     
#     bleu_base = sacrebleu.corpus_bleu(base_pred, [refs]).score
#     bleu_ours = sacrebleu.corpus_bleu(ours_pred, [refs]).score
#     return bleu_base, bleu_ours

# bleu_easy_b, bleu_easy_o = eval_split(easy_df)
# bleu_hard_b, bleu_hard_o = eval_split(hard_df)

# labels = ["easy", "hard"]
# plt.bar([0,1],       [bleu_easy_b, bleu_hard_b], width=.35, label="baseline")
# plt.bar([0.35,1.35], [bleu_easy_o, bleu_hard_o], width=.35, label="ours")
# plt.xticks([0.17,1.17], labels)
# plt.ylabel("BLEU ↑")
# plt.title("mBART устойчивость на Reddit dev")
# plt.legend()
# plt.tight_layout()
# plt.show()

easy = 2347  |  hard = 718
Easy samples:


,en,ru,difficulty
0,I’ll prolly be picking up the i Sport!,"Наверное, я возьму iSport!",easy
1,"My mind just melted after seeing this, I reall...","Я просто растаял, увидев это, мне бы реально х...",easy
2,"Such a cute clitty, and pussy you have",У тебя такие милые клиторок и киска.,easy
3,Spider King in SKT - 2headed spider that has a...,"Король-паук в SKT–2 — это двухголовый паук, у ...",easy
4,"The usual recommendations are Mirai Nikki, Hig...","Обычно рекомендуют Mirai Nikki, Higurashi, Ano...",easy
...,...,...,...
125,Just unsheathe me and point me at the enemy.,Просто обнажи меня и направь на врага.,easy
126,But blocking payments to crypto has no effect ...,Но блокировка платежей на криптокошельки не вл...,easy
127,If the money in your Revolut account is illega...,Если деньги на твоем счете Revolut получены не...,easy
128,It's just to force you to buy crypto in Revolu...,"Это просто для того, чтобы принудить тебя поку...",easy


Hard samples:


,en,ru,difficulty
7,If you get denied (or ignored) by every commen...,Если тебе откажут (или тебя проигнорируют) все...,hard
8,ambery hazel,янтарно-карие,hard
10,"Alternatively, Niantic is handling moved porta...",Либо Niantic сейчас по-другому обрабатывает пе...,hard
16,He’s a fucking weeny for bringing it up!😂🤣😂🤣😂 ...,"Он гребаный сопляк, раз заговорил об этом! 😂🤣😂...",hard
17,"No Kiba is the harem member lol, it's a joke 😭","Нет, Киба — член гарема. Лол, это шутка 😭.",hard
...,...,...,...
410,You need Conan as iron man 😂,Вам нужен Конан в роли Железного человека 😂.,hard
412,Who’s quitting?!?!,Кто уходит?!?!,hard
414,"Keep driving the price down hedge fucks, I’ll ...","Продолжайте снижать цены, засранцы из хедж-фон...",hard
417,"But my gems are free earned from the game, i g...","Но мои гемы бесплатные, заработанные в игре, к...",hard


In [11]:


test_texts = [
    "Think of it as macros/python with a GUI to do a lot the work instead of writing code.",
    "how much did uy save up before leaving.",
    "Why We Sleep - This book is really eye-opening and really makes you evaluate your relationship with sleep.",
    "But I know DFV have diamond balls so if he exercises, he will hold that shares tight as fuck.",
    "Ive never heard of this.. Would it be wise to underfoot a 3070 or a 2070 super?",
    "2/8 and 2/24 and still waiting.🙄",
    "His JP enhancements were pretty terrible though, he should be a bit better in GL",
    "You will get lower hashrates, but your gui should be more usable.",
    "If you get enough critrate go for the atlas",
    "Yeah that makes sense, ty.",
    "I remember when I originally watched the film being completely sucked in, but it hasn't aged well.",
    "no he makes an eeeeee sound when u shoot we probably shot the door down on him and he is in pain plus the bullies didn't know what happens out of the school all they know is the signal tower the eye and the janitor",
    "Trash coin lol, I'm a lover of shitcoins but haven't touched bsc lol each to their own 😂",
    "The screams of pain in the background did not feel forced at all and added to the already superb vocals",
    "I am really lazy so finding this is a god send lmao.",
    "It's very intuitive & works really well, IMO.",
    "Yes",
    "My name is Oleg",
    "How are you?",
    "This model is not very good.",
    "A random sentence to check quality."
]

test_texts_ru = [
    "Представь себе, что это макрос/python с пользовательским интерфейсом, в котором ты можешь много работать вместо того, чтобы писать код.",
    "Сколько тебе удалось накопить перед тем, как ты ушел?",
    "Почему мы спим: это действительно познавательная книга, которая реально позволяет вам оценить ваши взаимоотношения со сном.",
    "Но я знаю, что у DFV алмазные яйца, и если он их реализует, он будет держать эти акции так крепко, что хрен оторвешь.",
    "Я об этом никогда не слышал... Имеет ли смысл делать андервольтинг на 3070 или 2070 Super?",
    "8.02, 24.02, и все еще жду.🙄",
    "Хотя его усовершенствования в JP были довольно ужасны, в GL ему должно быть немного получше.",
    "Ваш хешрейт будет ниже, но пользовательский интерфейс будет более удобным для использования.",
    "Если получишь достаточно CRIT-рейтинга, иди за атласом",
    "Да, в этом есть смысл, спасибо.",
    "Я помню, что, когда смотрел фильм в первый раз, меня совершенно засосало, но проверку временем он не прошел.",
    "Нет, он издает звук «иииии», когда ты стреляешь. Мы, наверное, проигнорировали его, и ему больно. Плюс, те, кто его травили, не знали о том, что происходит за пределами школы. Все, о чем они знают, это сигнальная башня, глаз и уборщик.",
    "Мусорный коин, лол. Мне нравятся говнокоины, но к BSC я не притрагивался, лол. Каждому свое 😂.",
    "Крики от боли на фоне совсем не казались имитацией и дополнили и без того превосходный вокал",
    "Я реально ленивый, поэтому найти это — прям божий дар, хаха.",
    "По-моему, это очень интуитивно понятный инструмент, который действительно хорошо работает.",
    "Да",
    "Мое имя Олег",
    "Как вы?",
    "Эта модель не очень хороша",
    "Случайное предложение для проверки качества"    
]

ru_texts_demo1 = [None] * len(test_texts)


print("Number of lines in texts:", len(test_texts))


# if not ru_texts_demo1:
#     for i, txt in enumerate(test_texts):

#         inputs = tokenizer(txt, return_tensors="pt", truncation=True, padding=True, max_length=128)
#         inputs = {key: value.to(device) for key, value in inputs.items()}


#         with torch.no_grad():
#             gen_ids = model.generate(
#                 **inputs,
#                 max_length=200,
#                 num_beams=5,  
#                 do_sample=False, 
#                 forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"], 
#                 repetition_penalty=1.1
#             )


#         ru_translation = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
#         ru_texts_demo1.append(ru_translation)


#         input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

#         del inputs, gen_ids

#     gc.collect()


#     with open(ru_texts_file, "w", encoding="utf-8") as f:
#         for line in ru_texts_demo:
#             f.write(line + "\n")

#     #####-----------------------------------------------


# print_gpu_memory_usage("After processing lines")

# test_texts.extend(hard_100["en"])
# test_texts_ru.extend(hard_100["ru"])

# # easy_100 = easy_df.iloc[:100][["en","ru","difficulty"]]
# # hard_100 = hard_df.iloc[:100][["en","ru","difficulty"]]

# scores_demo = extract_attention_uncertainty(test_texts, model, tokenizer, device)


# inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt').to(device)


# (translated_ids1, scores_dropout1,
#  entropy_scores_demo1, attention_variances_demo1,
#  hidden_variances_demo1) = mc_dropout_inference(model, inputs, n=10, batch_size=1, device=device)

# # print("scores_dropout1 ", scores_dropout1)
# # print("energy_scores_demo1 ", energy_scores_demo1)
# # print("entropy_scores_demo1 ", entropy_scores_demo1)
# # print("attention_variances_demo1 ", attention_variances_demo1)
# # print("hidden_variances_demo1 ", hidden_variances_demo1)






# for i, text in enumerate(test_texts):
#     
#     inputs = tokenizer(text, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_length=200,
#             num_beams=10,
#             do_sample=False,
#             forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
#             repetition_penalty=1.1
#         )
#     original_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     
#     tgt_text = test_texts_ru[i]

#    
#     refined_translation = refine_translation_with_qe(
#         src_text=text,
#         tgt_text=tgt_text,
#         mbart_model=model,
#         qe_model=qe_model,
#         tokenizer=tokenizer,
#         device=device
#     )


test_texts.extend(hard_100["en"])
test_texts_ru.extend(hard_100["ru"])




decode_kwargs = dict(
    max_length=200, num_beams=4, do_sample=True,
    top_k=50, top_p=0.9, temperature=0.8,
    num_return_sequences=1,
    forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"],
    repetition_penalty=1.1
)
threshold = 0.9


benchmark = {
    "no_adapt":   {"train_gpu_h": 0.0, "epoch_times": [], "peak_mem": 0, "infer_times": []},
    "proposed":   {"train_gpu_h": 0.0, "epoch_times": [], "peak_mem": 0, "infer_times": []},
    "full_finet": {"train_gpu_h": 0.0, "epoch_times": [], "peak_mem": 0, "infer_times": []},
}
def record_memory(tag):
    if torch.cuda.is_available():
        used = torch.cuda.max_memory_reserved() / (1024**3)
        benchmark[tag]["peak_mem"] = max(benchmark[tag]["peak_mem"], used)


torch.cuda.reset_peak_memory_stats()
start_inf = time.time()
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        _ = model.generate(
            **inputs,
            max_length=200, num_beams=10,
            forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
        )
end_inf = time.time()
benchmark["no_adapt"]["infer_times"].append((end_inf - start_inf) / len(test_texts))
record_memory("no_adapt")



RISK_BUFFER = []
for src, tgt in zip(test_texts, test_texts_ru):

    inputs = tokenizer(src, return_tensors="pt").to(device)
    with torch.no_grad():
        _ = model.generate(**inputs, **decode_kwargs)

    attn_entropy = extract_attention_uncertainty([src], model, tokenizer, device)[0].item()
    feat = extract_mbart_features(src, model, tokenizer, device)
    with torch.no_grad():
        qe_unc = qe_model(feat, torch.zeros(1,4, device=device)).item()
    risk = 0.7 * attn_entropy + 0.3 * qe_unc
    if risk > threshold:
        RISK_BUFFER.append((src, tgt))


torch.cuda.reset_peak_memory_stats()
t0 = time.time()
prop_epochs = full_finetune(
    model, tokenizer, device,
    all_pairs=RISK_BUFFER,
    epochs=2,
    batch_size=8,
    lr=1e-5,
    lbl_smth=0.1
)
dt = time.time() - t0
benchmark["proposed"]["train_gpu_h"] = dt / 3600
benchmark["proposed"]["epoch_times"] = prop_epochs
record_memory("proposed")


torch.cuda.reset_peak_memory_stats()
start_inf = time.time()
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        _ = model.generate(**inputs, **decode_kwargs)
end_inf = time.time()
benchmark["proposed"]["infer_times"].append((end_inf - start_inf) / len(test_texts))
record_memory("proposed")


torch.cuda.reset_peak_memory_stats()
t0 = time.time()
full_epochs = full_finetune(
    model, tokenizer, device,
    all_pairs=list(zip(test_texts, test_texts_ru)),
    epochs=2,
    batch_size=10,
    lr=1e-5,
    lbl_smth=0.1
)
dt_full = time.time() - t0
benchmark["full_finet"]["train_gpu_h"] = dt_full / 3600
benchmark["full_finet"]["epoch_times"] = full_epochs
record_memory("full_finet")


start_inf = time.time()
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        _ = model.generate(
            **inputs,
            max_length=200, num_beams=10,
            forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
        )
end_inf = time.time()
benchmark["full_finet"]["infer_times"].append((end_inf - start_inf) / len(test_texts))
record_memory("full_finet")


rows = []
name_map = {
    "no_adapt":   "Без адаптации",
    "proposed":   "Предлагаемый метод",
    "full_finet": "Полный fine-tuning"
}
for method, stats in benchmark.items():

    if stats["epoch_times"]:
        avg_min = int((sum(stats["epoch_times"]) / len(stats["epoch_times"])) // 60)
        epoch_str = f"{avg_min} мин"
    else:
        epoch_str = "—"

    gpu_h = f"{stats['train_gpu_h']:.1f}" if method != "no_adapt" else "—"

    mem = f"{stats['peak_mem']:.1f}"

    inf = f"{stats['infer_times'][0]*1000:.0f}" if stats["infer_times"] else "—"

    rows.append({
        "Метод":                name_map[method],
        "GPU-часы":            gpu_h,
        "Время эпохи":         epoch_str,
        "Память (ГБ)":         mem,
        "Инференс (мс/пример)": inf
    })

df_bench = pd.DataFrame(rows)
print(df_bench.to_markdown(index=False))    


    # print(f"--- Sample {i} ---")
    # print(f"Input: {text}\n")
    # print(f"Reference Translation: {tgt_text}\n")
    # print(f"Original Translation (MBart): {original_translation}\n")
    # print(f"Refined Translation (MBart + QE): {refined_translation}\n\n")




    # inputs = tokenizer(text, return_tensors="pt").to(device)
    # with torch.no_grad():

    #     outputs = model(**inputs, output_attentions=True, return_dict=True)

    # attentions = outputs.encoder_attentions 
    # avg_attention = torch.mean(torch.stack(attentions), dim=(0, 1)) 
    # attn_entropy = -torch.sum(avg_attention * torch.log(avg_attention + 1e-9), dim=-1).mean()


    # score_mcdropout1 = scores_dropout1[i].item() if i < len(scores_dropout1) else None

    # energy_score_value1 = energy_scores_demo1[i] if i < len(energy_scores_demo1) else None


    # entropy_value1 = entropy_scores_demo1[i] if i < len(entropy_scores_demo1) else None


    # avg_variance = attention_variances_demo1[i].mean().item()
    # specific_variance = attention_variances_demo1[i].item()


    # input_ids = inputs["input_ids"].float() 


    # mean_input_ids = torch.mean(input_ids, dim=0)  
    # cov_input_ids = torch.cov(input_ids.T) 


    # cosine_similarity_value = cosine_similarity_score(input_ids, mean_input_ids.unsqueeze(0))

    # print(f"Uncertainty via attention entropy (from QEDataset): {attn_entropy:.4f}")
    # print()
    # print(f"Attention Variance (mean): {avg_variance:.4f}")
    # print()
    # print(f"Attention Variance (index 0): {specific_variance:.4f}")
    # print()

    # if score_mcdropout1 is not None:
    #     print(f"Uncertainty via MC Dropout (from QEDataset): {score_mcdropout1:.4f}")
    #     print()
    # else:
    #     print("Warning: MC Dropout Score is missing.")
        
    # if energy_score_value1 is not None:
    #     print(f"Energy Score: {energy_score_value1:.4f}")
    #     print()
    # else:
    #     print("Warning: Energy Score is missing.")
        
    # if entropy_value1 is not None:
    #     print(f"Entropy Score: {entropy_value1:.4f}")
    #     print()
    # else:
    #     print("Warning: Entropy Score is missing.")
    # print("=" * 80)

Number of lines in texts: 21


NameError: name 'full_finetune' is not defined

In [ ]:


print("Вывод содержимого QEDataset:")
start_index = 0
for i in range(start_index, min(30, len(dataset_demo_mcdropout))):
    en_text, ru_text, score_value = dataset_demo[i]
    en_text_mcdropout, ru_text_mcdropout, score_tensor_mcdropout = dataset_demo_mcdropout[i]
    score_value_mcdropout = score_tensor_mcdropout.item()

    refined_translation = refine_translation_with_qe(en_text, ru_text, model, qe_model, tokenizer, device)

    print(f"--- Образец {i} ---")
    print(f"English: {en_text}")
    print()
    print()
    print(f"Original Translation (ru_lines): {ru_lines[i]}")
    print()
    print()
    print("-" * 100)
    print()
    print(f"Refined Translation (MBart + QE): {refined_translation}")
    
    print("-" * 100)
    print()
    print()
    print(f"Uncertainty через энтропию внимания (из QEDataset): {score_value:.4f}")
    print()
    print(f"Uncertainty через MC Dropout (из QEDataset): {score_value_mcdropout:.4f}")
    print()

    # if i < len(energy_scores_demo):
    #     print(f"Energy Score: {energy_scores_demo[i]:.4f}")
    #     print()
    # else:
    #     print("⚠ Warning: Energy Score не найден для этого примера.")
        

    if i < len(entropy_scores_demo):
        print(f"Entropy Score: {entropy_scores_demo[i]:.4f}")
        print()
    else:
        print("⚠ Warning: Entropy Score не найден для этого примера.")


    avg_variance = attention_variances_demo[i].mean().item()
    print(f"Attention Variance (mean): {avg_variance:.4f}")
    print()

    desired_index = 0
    specific_variance = attention_variances_demo[i].item()
    print(f"Attention Variance (index {desired_index}): {specific_variance:.4f}")
    print()



    input_tensor = inputs["input_ids"].float()

    if input_tensor.shape[0] < 4:
        noise = torch.randn((4 - input_tensor.shape[0], input_tensor.shape[1]), device=input_tensor.device) * 1e-6
        input_tensor = torch.cat([input_tensor, noise], dim=0)
    

    if i >= input_tensor.shape[0]:
        cosine_similarity_value = torch.tensor(0.0, dtype=torch.float32, device=device)  # Теперь это тензор
    else:
        h = input_tensor[i].unsqueeze(0) if input_tensor[i].dim() == 1 else input_tensor[i]
        h_train = torch.mean(input_tensor.float(), dim=0, keepdim=True)
        cosine_similarity_value = cosine_similarity_score(h, h_train)

    print(f"Cosine Similarity (OOD Score): {cosine_similarity_value:.4f}")

    
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
